In [ ]:
%matplotlib ipympl

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.signal import welch
import gsw

In [ ]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [ ]:
ds

In [ ]:
plt.figure()
plt.plot(np.mean(ds['C'],axis=0),ds['binheight'],'-o')
plt.title('correlation [128 = high signal/noise]')
plt.ylabel('height above instrument')

plt.figure()
plt.plot(np.mean(ds['EA'],axis=0),ds['binheight'],'-o')
plt.title('echo amplitude [dB]')
plt.ylabel('height above instrument')

In [ ]:
bini = np.arange(1,17)

plt.figure()
plt.plot(np.mean(ds['Eas'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.plot(np.mean(ds['Nor'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.title('mean velocity - entire deployment')
plt.legend(['East','North'])
plt.ylabel('height above instrument')
plt.xlabel('[m/s]')

In [ ]:
dt = 5*60 # 5 min sample interval (in seconds)

x = cumtrapz(ds['Eas']*dt,axis=0)
y = cumtrapz(ds['Nor']*dt,axis=0)

In [ ]:
plt.figure()
plt.plot(x[:,15]/1000,y[:,15]/1000)
plt.plot(x[:,7]/1000,y[:,8]/1000)
plt.plot(x[:,1]/1000,y[:,1]/1000)
plt.plot(0,0,'ko')
plt.axis('equal')
plt.xlabel('distance east [km]')
plt.ylabel('distance north [km]')
plt.legend(['Bin 15','Bin 8','Bin 1'])

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]);
plt.plot(ds['time'],ds['Eas'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('eastward velocity')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_filt'][:,15]);
plt.plot(ds['time'],ds['Eas_filt'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('eastward velocity')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_filt'][:,15]);
plt.plot(ds['time'],ds['Nor_filt'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

In [ ]:
from graph import stick_plot



fig, ax = plt.subplots(figsize=(8, 2.25))

q = stick_plot(np.array(ds['time'])[::288],
               np.array(ds['Eas_filt'])[::288,15],
               np.array(ds['Nor_filt'])[::288,15],
               ax=ax,width=0.0015)

ref = 0.01
qk = plt.quiverkey(q, 0.1, 0.85, ref,
                  "%s m s$^{-1}$" % ref,
                  labelpos='N', coordinates='axes')
_ = plt.xticks(rotation=0)

plt.title('Station M - low pass filtered velocity - bin 15')

In [ ]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]);
plt.ylabel('m/s')
plt.xlim([t1,t2])
plt.ylim([-0.15,0.15])
plt.legend(['East','North'])
plt.title('full velocity, '+str(t1)+'-'+str(t2)+', bin 15')

In [ ]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]-ds['Eas_tide'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]-ds['Nor_tide'][:,15]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.ylim([-0.15,0.15])
plt.xlim([t1,t2])
plt.legend(['East','North'])
plt.title('detided velocity, '+str(t1)+'-'+str(t2)+', bin 15')

In [ ]:
Eas = np.array(ds['Eas'])
Nor = np.array(ds['Nor'])
Eas_tide = np.array(ds['Eas_tide'])
Nor_tide = np.array(ds['Nor_tide'])

N = len(Eas)
fE,SE = welch(Eas[:,5],fs=288.,nperseg=N/6,window='hanning')
fN,SN = welch(Nor[:,5],fs=288.,nperseg=N/6,window='hanning')

#rotary spectrum
fr,Sr = welch(Eas[:,5]+1j*Nor[:,5],
              fs=288.,nperseg=N/6,
              window='hann',return_onesided=False)
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

In [ ]:
f_cpd = gsw.f(35+8.4585/60)*86400/(2*np.pi)

plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra')

In [ ]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')

yl = plt.ylim()
plt.plot([f_cpd,f_cpd],yl,'-',color='gray')
plt.plot([24/25.82,24/25.82],yl,'-',color='gray')
plt.plot([24/23.93,24/23.93],yl,'-',color='gray')
plt.plot([2,2],yl,'-',color='gray')
plt.plot([24/12.42,24/12.42],yl,'-',color='gray')
plt.ylim(yl)
plt.xlim([0.5,5])

plt.text(f_cpd,yl[1],'$f$',fontsize=12,
         horizontalalignment='center',
         verticalalignment='bottom')
plt.text(24/12.42,yl[1],'$M_2$',fontsize=12,
         horizontalalignment='right',
         verticalalignment='bottom')
plt.text(2,yl[1],'$S_2$',fontsize=12,
         horizontalalignment='left',
         verticalalignment='bottom')
plt.text(24/23.93,yl[1],'$K_1$',fontsize=12,
         horizontalalignment='left',
         verticalalignment='bottom')
plt.text(24/25.82,yl[1],'$O_1$',fontsize=12,
         horizontalalignment='right',
         verticalalignment='bottom')